<a href="https://colab.research.google.com/github/Mehmetakiftasoz/makif2/blob/MAKIF_COLAB/benzerlik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [190]:
!pip install rapidfuzz

!pip install fuzzywuzzy

In [191]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import re
# Install fuzzywuzzy
!pip install fuzzywuzzy # This line installs the missing fuzzywuzzy package.
from fuzzywuzzy import process, fuzz # Now, you can import the module.

from google.colab import files


In [192]:
df_me5a = pd.read_csv('/content/ME5A_2022-2025.csv', sep=None, engine='python')
df_1 = pd.read_csv('/content/Приложение_1.csv', sep=None, engine='python')
df_2 = pd.read_csv('/content/Приложение_2.csv', sep=None, engine='python')
df_3 = pd.read_csv('/content/Приложение_3.csv', sep=None, engine='python')
df_4 = pd.read_csv('/content/Приложение_4.csv', sep=None, engine='python')
df_5 = pd.read_csv('/content/Приложение_5.csv', sep=None, engine='python')
df_6 = pd.read_csv('/content/Приложение_6.csv', sep=None, engine='python')
df_7 = pd.read_csv('/content/Приложение_7.csv', sep=None, engine='python')
df_8 = pd.read_csv('/content/Приложение_8.csv', sep=None, engine='python')
df_9 = pd.read_csv('/content/Приложение_9.csv', sep=None, engine='python')
df_2_1 = pd.read_csv('/content/Приложение_2_1.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases

In [193]:
df_1.head(5)

,﻿№ п/п,Номенклатура
0,1,"Труба Х-76,1х8,8-08Х18Н10Т-2-ГОСТ Р 70731.2-2023"
1,2,Труба бесшовная из коррозионно-стойкой стали 3...
2,3,Труба бесшовная горячедеформированная из корро...
3,4,"Труба Х-42,4х5,0-08Х18Н10Т-2-ГОСТ Р 70731.2-2023"
4,5,"Труба Х-168,3х5,0-15ХМ-2 ГОСТ Р 70731.1-2023"


In [194]:
df_me5a.head(5)

,﻿Заявка,Позиция заявки,Материал,Индикатор удаления,Затребовал,Индик. выдачи,Дата заявки,Заказ на поставку,Позиция ЗкзНаПостав,Единица измерения,RU Наименование
0,2200000406,10,4.500131e+09,YANLIŞ,Poberezhnik,2,12.11.2022,3.100002e+09,240,ШТ,"НАКОНЕЧНИК Ф.""КВТ"" МЕДНЫЙ ЛУЖЁННЫЙ ПОД ОПРЕССО..."
1,2200000406,20,4.500142e+09,YANLIŞ,Poberezhnik,2,12.11.2022,3.100011e+09,20,КГ,ПОЛОСА СТАЛЬНАЯ 4Х25 ГОСТ 103-2006 СТ3
2,2200000406,30,4.500082e+09,YANLIŞ,Poberezhnik,2,12.11.2022,4.600005e+09,580,М,МЕТАЛЛОРУКАВ ГЕРМЕТИЧНЫЙ В ПВХ ОБОЛОЧКЕ РЗ-ЦПН...
3,2200000406,40,4.500138e+09,YANLIŞ,Poberezhnik,2,12.11.2022,3.100005e+09,800,М,МЕТАЛЛОРУКАВ МРН 20 ТУ 25.99.29-072-97284872-2017
4,2200000406,50,4.500039e+09,YANLIŞ,Poberezhnik,2,12.11.2022,NaN,0,ШТ,"МУФТА ВВОДНАЯ ВМ20 IP54 АРТ.61370 Ф.""FORTISFLEX"""


In [195]:
print(df_me5a.columns)


Index(['﻿Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления',
       'Затребовал', 'Индик. выдачи', 'Дата заявки', 'Заказ на поставку',
       'Позиция ЗкзНаПостав', 'Единица измерения', 'RU Наименование'],
      dtype='object')


In [196]:
# 🟦 Adım 1: df_1'in kopyası
df_1_active = df_3.copy()

In [197]:
# 🟦 Adım 2: df_me5a'nın kopyası ve filtrelenmesi
df_me5a_active = df_me5a.copy()
df_me5a_active = df_me5a_active[df_me5a_active['Заказ на поставку'].isna()].reset_index(drop=True)

In [198]:
df_me5a_active['Заказ на поставку'].unique()

array([nan])

In [199]:
# 🧹 Sütun adlarındaki boşlukları ve görünmeyen karakterleri temizle
df_me5a.columns = df_me5a.columns.str.strip()
df_me5a_active.columns = df_me5a_active.columns.str.strip()
df_1.columns = df_1.columns.str.strip()
df_1_active.columns = df_1_active.columns.str.strip()


In [200]:
print(df_me5a_active.columns)

Index(['﻿Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления',
       'Затребовал', 'Индик. выдачи', 'Дата заявки', 'Заказ на поставку',
       'Позиция ЗкзНаПостав', 'Единица измерения', 'RU Наименование'],
      dtype='object')


alltaki çalışıyor ama hatalar var

In [ ]:
df_1_active = df_3.copy()
df_me5a_active = df_me5a[df_me5a['Заказ на поставку'].isna()].copy()

# 3. Sonuçları depolamak için boş listeler
zayavka_list = []
zatreboval_list = []
poz_list = []
unit_list = []
ru_name_list = []
score_list = []

# 4. Eşleştirme işlemi: Sadece %70 ve üstü olanları ekle
for nom in df_1_active['Номенклатура']:
    best_match = process.extractOne(
        nom,
        df_me5a_active['RU Наименование'],
        scorer=fuzz.token_sort_ratio
    )

    if best_match is not None and best_match[1] >= 20:
        matched_name = best_match[0]
        score = best_match[1]

        matched_row = df_me5a_active[df_me5a_active['RU Наименование'] == matched_name].iloc[0]

        zayavka_list.append(matched_row['﻿Заявка'])
        zatreboval_list.append(matched_row['Затребовал'])
        # Uncomment the following line and make sure the column name is correct
        poz_list.append(matched_row['Позиция ЗкзНаПостав'])
        unit_list.append(matched_row['Единица измерения'])
        ru_name_list.append(matched_row['RU Наименование'])
        score_list.append(score)
    else:
        # ❌ Eşleşme yoksa tamamen boş bırak
        zayavka_list.append(pd.NA)
        zatreboval_list.append(pd.NA)
        # Uncomment the following line and make sure the column name is correct
        poz_list.append(pd.NA)
        # Eşleşme olmadığında unit_list'e de pd.NA ekleyin
        unit_list.append(pd.NA)
        ru_name_list.append(pd.NA) # Eşleşme olmadığında ru_name_list'e de pd.NA ekleyin
        score_list.append(pd.NA)  # Eşleşme olmadığında score_list'e de pd.NA ekleyin


df_1_active['Заявка'] = zayavka_list
df_1_active['Затребовал'] = zatreboval_list
df_1_active['Позиция ЗкзНаПостав'] = poz_list
df_1_active['Единица измерения'] = unit_list
df_1_active['RU Наименование'] = ru_name_list
df_1_active['Benzerlik Skoru'] = score_list

In [ ]:
df_1_active[df_1_active['Benzerlik Skoru'] >=50 ]

In [ ]:
df_1_active.to_excel("deneme.xlsx", index=False)

In [ ]:
files.download("deneme.xlsx")